# All the functions relates to KNN (cosine similarity ) from different aspects

In [4]:
def Average(lst): 
    return sum(lst) / len(lst) 

In [5]:
def search_rating_user_id(userid,movieid,time):
    """
    Input
    ---
    userid:int, represent userid
    movieid:int, represent movieid (closest id that rated by given user)
    time:bin_time"""
    return rating[(rating['userId'] == userid) & (rating['movieId'] == movieid) &(rating['bin_num'] == time)]['rating']

In [6]:
def q_given_movieid(df,l_movieid):
    """
    Input
    ----
        df:q df,latent values of movies
        l_movieid:list of movie id, all element should be str 
    Return
    ----
        a dataframe that only contain given movieids"""
    
    return df[l_movieid]

In [7]:
def cosine_similarity (test_movie_id,userid):
    """
    Global variable
    ------
        data_train: train part of rating.csv
        
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        user_id: user id,int
    Return
    
    ----
        """
    #############
    #list of movie id for given user in train,int
    train_movieid_given_user = movieId_by_urserId(userid,data_train)
    #list of movie id for given user in train,str
    train_movieid_given_user_str = [str(val) for val in train_movieid_given_user]
    
    
    ###########
    
    
    l_latent_movie = []
    movie_cos_dic = {}
    for movie in train_movieid_given_user_str:
        #print(movie,type(movie)) #2，str

        numerator =array([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist]).dot(array([val
                      for sublist in np.asarray(q[[movie]]).tolist()
                      for val in sublist]))

        cosin_s = numerator/(norm([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist])*norm([val
                      for sublist in np.asarray(q[[test_movie_id]]).tolist()
                      for val in sublist]))
        l_latent_movie.append(cosin_s)# cosimilarity value for test latent vs all train latent
        
        dict_movieid_cos = dict(zip(train_movieid_given_user_str,l_latent_movie))
        l_sorted_s_to_l = sorted(dict_movieid_cos.items(), key=itemgetter(1)) #from small to large
        
    return [ele for ele in reversed(l_sorted_s_to_l)] 

In [8]:
def knn_k_user_base(k,test_movie_id,userid,df_mat):
    """
    Input
    -----
        k: int, knn's k
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        userid: user id,int
        df_mat: matrix after multiple with bias #9724 rows × 610 columns
    Return
    -----
        a list of 23 predicted rating for given movie id for given user 
        
        """
    distance_pair = cosine_similarity(test_movie_id,userid)
    
    l_movie_id = []
    for i in range(k):
        l_movie_id.append(int(distance_pair[i][0])) #str to int
        
#     l_user_id = [userid]*k
    
#     knn_pair = list(zip(l_user_id,l_movie_id))
    
#     for pair in knn_pair:
    l_rate_mean = []
    for i in range(23):
        l_rate = []
        for movie in l_movie_id:
        
            l_rate.append(df_mat[userid][movie][i])
        
        l_rate_mean.append(round(Average(l_rate),1))

    
    return l_rate_mean

In [9]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [10]:
def knn_user_base_campare(test_movie_id,userid, time,df_mat,k = 1):
    knn_result =  knn_k_user_base(k,test_movie_id,userid,df_mat)
    real_rating = search_rating_user_id(userid,int(test_movie_id),time)
    rmse_result = rmse(array(knn_result), real_rating.values)
    return rmse_result

In [11]:
def knn_campare(test_movie_id,df_mat,time):
    knn_result =  knn_k_(k,test_movie_id,userid,df_mat)
    real_rating = search_rating_user_id(userid,int(test_movie_id),time)
    rmse_result = rmse(array(knn_result), real_rating.values)
    return rmse_result

In [12]:
def cosine_similarity_all(test_movie_id):
    """
    Global variable
    ------
        data_train: train part of rating.csv
        
    Input
    -----
        test_movie_id: the id of movie that want to test,int. for testing purpose should also match same user
    Return
    
    ----
        """
  
    ###########
    movie_test_id_str_list = list(str(test_movie_id))

    movie_compare_id = [item for item in movie_id if item not in movie_test_id_str_list] #movie_id list without test id
    ###############

    l_latent_movie = []
    movie_cos_dic = {}
    for i in movie_compare_id:
       # print(i,type(i)) 2，str

        numerator =array([val
                      for sublist in np.asarray(q[[movie_test_id_str_list[0]]]).tolist()
                      for val in sublist]).dot(array([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))

        cosin_s = numerator/(norm([val
                      for sublist in np.asarray(q[[movie_test_id_str_list[0]]]).tolist()
                      for val in sublist])*norm([val
                      for sublist in np.asarray(q[[i]]).tolist()
                      for val in sublist]))
        l_latent_movie.append(cosin_s)


        dict_movieid_cos = dict(zip(movie_compare_id,l_latent_movie))
        l_sorted_s_to_l = sorted(dict_movieid_cos.items(), key=itemgetter(1)) #from small to large

    return ([ele for ele in reversed(l_sorted_s_to_l)])



In [13]:
def mat_check_rate(test_movie_id,df,userid,time):
    
    user_index = b_user_dic.get(str(userid))
    time_index = time - 1
    movie_index =b_item_dic.get(str(test_movie_id))
    rating_predict = df[user_index][movie_index][time_index]
    return round(rating_predict,1)

In [14]:
def knn_k_all(test_movie_id,df,userid,time):
    """
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        df: df_mat
        userid： int
        time: bin_num
    Return
    -----
        a list of  predicted rating for given movie id for given user given year of matrix
        
        """
    distance_pair = cosine_similarity_all(test_movie_id)

    movie_index = b_item_dic.get(distance_pair[0][0])
    user_index = b_user_dic.get(str(userid))
    time_index = time - 1
    rating_predict = df[user_index][movie_index][time_index]
    
    return round(rating_predict,1)

In [15]:
def knn_k_pq(test_movie_id,df,userid):
    """
    Input
    -----
        test_movie_id: the id of movie that want to test,str. for testing purpose should also match same user
        df: df_mat（only pq)
        userid： int
    Return
    -----
        a list of  predicted rating for given movie id for given user given year of matrix
        
        """
    distance_pair = cosine_similarity_all(test_movie_id)

    movie_index = b_item_dic.get(distance_pair[0][0])
    user_index = b_user_dic.get(str(userid))
    
    rating_predict = df[user_index][movie_index]
    
    return rating_predict

In [16]:
def pq_check_rate(test_movie_id,df,userid):
    """
    Input
    ----
        df: df_pq"""
    
    user_index = b_user_dic.get(str(userid))
    
    movie_index =b_item_dic.get(str(test_movie_id))
    
    rating_predict = df[user_index][movie_index]
    return rating_predict

In [155]:
def item_check_value(test_movie_id,mat):
    """
    Input
    ----
        mat: b_item,matrix
    Return
    -----
        a float"""
    
    #user_index = b_user_dic.get(str(userid))
    
    movie_index =b_item_dic.get(str(test_movie_id))
    
    value_predict = mat[0][movie_index]
    return value_predict

In [157]:
def user_check_value(userid,mat):
    """
    Input
    ----
        mat: b_user,matrix"""
    
    user_index = b_user_dic.get(str(userid))
    
    #movie_index =b_item_dic.get(str(test_movie_id))
    
    value_predict = mat[0][user_index]
    return value_predict

In [220]:
def time_check_value(test_movie_id,df,time_bin):
    """
    Input
    ----
        mat: b_time"""
    #print(test_movie_id,time_bin)
    time_index = b_time_dic.get(str(time_bin))
    #print(time_bin,time_index)
    
    movie_index =b_item_dic.get(str(test_movie_id))
    
    
    rating_predict = df[time_index][movie_index]
    return rating_predict

In [17]:
def pariwise_cosin(q):
    """
    Input
    -----
        q: q.csv
    Return
    -----
        """
    
#     q_matrix = q.copy()
#     q_matrix = q_matrix.values
    
#     q_knn_matrix = q_matrix.T
    
    return 1-pairwise_distances(q.values.T, metric="cosine")

In [18]:
#dist_result[9723]

In [19]:
#sorted(list(set(dist_result[9723].flatten().tolist())))[-2]

In [20]:
def pair_cos_id (test_movie_id):
    """
    Input
    -----
        test_movie_id:int
        pair_cos_q: nparray, pair_cosin_q (global var)
    Return
    -----
        list with index of nearst movie,id of nearest movie"""
    test_movie_index = b_item_dic.get(str(test_movie_id))
    df_cos = pd.DataFrame(pair_cos_q)
    
    
    high_2_info = df_cos[test_movie_index].nlargest(2)
    knn_near_index = np.where(df_cos[test_movie_index] == high_2_info.values[1])[0][0]
    
    #knn_movie_id = b_item_dic.get(str(test_movie_id))
    knn_movie_id  = list(b_item_dic.keys())[list(b_item_dic.values()).index(knn_near_index)] 
    #[knn_near_index,int(knn_movie_id)]
    return int(knn_movie_id)

In [48]:
def avg_rating_train_look_up(movie_id_int):
    """
    Global variable
    ------
        df_data_train:after groupby mean and reset index
        
    Return
    ------
        rating"""
    #print(df_data_train[df_data_train['movieId'] == movie_id_int])
    
    return rating[rating['movieId'] == movie_id_int]['rating'].values[0]

In [22]:
def main_get_rating_knn(test_movie_id):
    """
    Global var
    ------
        pair_cos_q
        df_data_train
    
    Return
    -----
        Get KNN rating """
    nearest_movie_id = pair_cos_id(test_movie_id)
    #print(nearest_movie_id)
    avg_rating_nearest_movie_id = avg_rating_train_look_up(nearest_movie_id)
    #print(avg_rating_nearest_movie_id)
    
    
    return avg_rating_nearest_movie_id
    

In [235]:
def b_time_user(user_id):
    rating_user = rating[rating['userId'] == user_id]
    rating_group = rating_user[['userId','movieId','bin_num']].values
    
    user_time_movie_list = []
    for j, pair in enumerate(rating_group):
        user_time_movie_list.append((time_check_value(pair[1],b_time,pair[2])))
    return user_time_movie_list
    #time_check_value(test_movie_id,df,time_bin)
    #b_item_col_qiqi.append(b_item_col_qiqi)
